In [1]:
import numpy as np
import pandas as pd

from collections import Counter

In [2]:
def construir_arbol(instancias, etiquetas):
    # ALGORITMO RECURSIVO para construcción de un árbol de decisión binario. 
    
    # Suponemos que estamos parados en la raiz del árbol y tenemos que decidir cómo construirlo. 
    ganancia, pregunta = encontrar_mejor_atributo_y_corte(instancias, etiquetas)
    
    # Criterio de corte: ¿Hay ganancia?
    if ganancia == 0:
        #  Si no hay ganancia en separar, no separamos. 
        return Hoja(etiquetas)
    else: 
        # Si hay ganancia en partir el conjunto en 2
        instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen = partir_segun(pregunta, instancias, etiquetas)
        # partir devuelve instancias y etiquetas que caen en cada rama (izquierda y derecha)

        # Paso recursivo (consultar con el computador más cercano)
        sub_arbol_izquierdo = construir_arbol(instancias_cumplen, etiquetas_cumplen)
        sub_arbol_derecho   = construir_arbol(instancias_no_cumplen, etiquetas_no_cumplen)
        # los pasos anteriores crean todo lo que necesitemos de sub-árbol izquierdo y sub-árbol derecho
        
        # sólo falta conectarlos con un nodo de decisión:
        return Nodo_De_Decision(pregunta, sub_arbol_izquierdo, sub_arbol_derecho)

In [3]:
# Definición de la estructura del árbol. 

class Hoja:
    #  Contiene las cuentas para cada clase (en forma de diccionario)
    #  Por ejemplo, {'Si': 2, 'No': 2}
    def __init__(self, etiquetas):
        self.cuentas = dict(Counter(etiquetas))


class Nodo_De_Decision:
    # Un Nodo de Decisión contiene preguntas y una referencia al sub-árbol izquierdo y al sub-árbol derecho
     
    def __init__(self, pregunta, sub_arbol_izquierdo, sub_arbol_derecho):
        self.pregunta = pregunta
        self.sub_arbol_izquierdo = sub_arbol_izquierdo
        self.sub_arbol_derecho = sub_arbol_derecho
        
        
# Definición de la clase "Pregunta"
class Pregunta:
    def __init__(self, atributo, valor):
        self.atributo = atributo
        self.valor = valor
    
    def cumple(self, instancia):
        # Devuelve verdadero si la instancia cumple con la pregunta
        return instancia[self.atributo] == self.valor
    
    def __repr__(self):
        return "¿Es el valor para {} igual a {}?".format(self.atributo, self.valor)

In [31]:
def gini(etiquetas):
    p = np.sum(etiquetas=='Si') / len(etiquetas)
    # impureza = 1 - p**2 - (1 - p)**2
    impureza = (p) * (1 - (p)) + (1 - p) * (1 - (1 - p))
    # impureza = 1 - sum([(cantidad/len(etiquetas))**2 for cantidad in Counter(etiquetas).values()])
    return impureza

def ganancia_gini(instancias, etiquetas_rama_izquierda, etiquetas_rama_derecha):
    gini_izq = (len(etiquetas_rama_izquierda) / len(instancias)) * gini(etiquetas_rama_izquierda)
    gini_der = (len(etiquetas_rama_derecha) / len(instancias)) * gini(etiquetas_rama_derecha)
    ganancia_gini = gini(etiquetas_rama_izquierda.append(etiquetas_rama_derecha)) - (gini_izq + gini_der)
    return ganancia_gini


def partir_segun(pregunta, instancias, etiquetas):
    # Esta función debe separar instancias y etiquetas según si cada instancia cumple o no con la pregunta (ver método 'cumple')
    cumplen = pregunta.cumple(instancias)

    instancias_cumplen = instancias[cumplen]
    etiquetas_cumplen = pd.Series(etiquetas)[cumplen]
    
    instancias_no_cumplen = instancias[~cumplen]
    etiquetas_no_cumplen = pd.Series(etiquetas)[~cumplen]
    
    return instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen

In [32]:
def encontrar_mejor_atributo_y_corte(instancias, etiquetas):
    max_ganancia = 0
    mejor_pregunta = None
    for columna in instancias.columns:
        for valor in set(instancias[columna]):
            # Probando corte para atributo y valor
            pregunta = Pregunta(columna, valor)
            _, etiquetas_rama_izquierda, _, etiquetas_rama_derecha = partir_segun(pregunta, instancias, etiquetas)

            ganancia = ganancia_gini(instancias, etiquetas_rama_izquierda, etiquetas_rama_derecha)
            
            if ganancia > max_ganancia:
                max_ganancia = ganancia
                mejor_pregunta = pregunta
    
    return max_ganancia, mejor_pregunta


def imprimir_arbol(arbol, spacing=""):
    if isinstance(arbol, Hoja):
        print (spacing + "Hoja:", arbol.cuentas)
        return

    print (spacing + str(arbol.pregunta))

    print (spacing + '--> True:')
    imprimir_arbol(arbol.sub_arbol_izquierdo, spacing + "  ")

    print (spacing + '--> False:')
    imprimir_arbol(arbol.sub_arbol_derecho, spacing + "  ")

In [33]:
X = pd.DataFrame([["Sol","Calor","Alta","Debil"],
                ["Sol","Calor","Alta","Fuerte"],
                ["Nublado","Calor","Alta","Debil"],
                ["Lluvia","Templado","Alta","Debil"],
                ["Lluvia","Frio","Normal","Debil"],
                ["Lluvia","Frio","Normal","Fuerte"],
                ["Nublado","Frio","Normal","Fuerte"],
                ["Sol","Templado","Alta","Debil"],
                ["Sol","Frio","Normal","Debil"],
                ["Lluvia","Templado","Normal","Debil"],
                ["Sol","Templado","Normal","Fuerte"],
                ["Nublado","Templado","Alta","Fuerte"],
                ["Nublado","Calor","Normal","Debil"],
                ["Lluvia","Templado","Alta","Fuerte"]],
                columns = ['Cielo', 'Temperatura', 'Humedad', 'Viento'])

y = ['No', 'No', 'Si', 'Si', 'Si', 'No', 'Si', 'No', 'Si', 'Si', 'Si', 'Si', 'Si', 'No']

In [34]:
arbol = construir_arbol(X, y)
imprimir_arbol(arbol)

0.6
0.6666666666666666
0.6428571428571429
1.0
0.5
0.6428571428571429
0.4
0.7777777777777778
0.6428571428571429
0.75
0.6
0.6428571428571429
0.6666666666666666
0.625
0.6428571428571429
0.5
0.7
0.6428571428571429
0.42857142857142855
0.8571428571428571
0.6428571428571429
0.8571428571428571
0.42857142857142855
0.6428571428571429
0.75
0.5
0.6428571428571429
0.5
0.75
0.6428571428571429
1.0
nan
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.6
0.4
0.5
0.4
0.6
0.5
0.6666666666666666
0.42857142857142855
0.5
0.6
0.4
0.5
0
0.625
0.5
0.2
0.8
0.5
0.8
0.2
0.5
0.6666666666666666
0.25
0.5
0.25
0.6666666666666666
0.5
0.5
0
0.2
0
0.5
0.2
0.3333333333333333
0
0.2
0
0.3333333333333333
0.2
0.2
nan
0.2
0.3333333333333333
0
0.2
0
0.3333333333333333
0.2
0.5
nan
0.5
0.5
nan
0.5
0.5
nan
0.5
1.0
0
0.5
0
1.0
0.5
1.0
nan
1.0
1.0
nan
1.0
1.0
nan
1.0
1.0
nan
1.0
0
nan
0
0
nan
0
0
nan
0
0
nan
0
0
nan
0
0
0
0
0
0
0
0
nan
0
0
0
0
0
0
0
0.6666666666666666
1.0
0.8
1.0
0.6666666666

## Resultado esperado

```
¿Es el valor para Cielo igual a Nublado?
--> True:
  ¿Es el valor para Temperatura igual a Frio?
  --> True:
    Hoja: {'Si': 1}
  --> False:
    ¿Es el valor para Temperatura igual a Templado?
    --> True:
      Hoja: {'Si': 1}
    --> False:
      Hoja: {'Si': 2}
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Fuerte?
    --> True:
      Hoja: {'No': 1, 'Si': 1}
    --> False:
      ¿Es el valor para Cielo igual a Sol?
      --> True:
        Hoja: {'Si': 1}
      --> False:
        Hoja: {'Si': 2}
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
      ¿Es el valor para Temperatura igual a Templado?
      --> True:
        Hoja: {'No': 1}
      --> False:
        Hoja: {'No': 2}
    --> False:
      Hoja: {'Si': 1, 'No': 1}
```

## Parte 2 (opcional)
Protocolo sklearn para clasificadores. Completar el protocolo requerido por sklearn. Deben completar la función predict utilizando el árbol para predecir valores de nuevas instancias. 


In [17]:
def predecir(arbol, x_t):
    if arbol.pregunta.cumple(x_t):
        arbol = arbol.sub_arbol_izquierdo
    else:
        arbol = arbol.sub_arbol_derecho
    
    if type(arbol) == Hoja:
        return np.sum(arbol.cuentas=='Si') / len(arbol.cuentas)

    return predecir(arbol, x_t)
        
class MiClasificadorArbol(): 
    def __init__(self):
        self.arbol = None
        self.columnas = ['Cielo', 'Temperatura', 'Humedad', 'Viento']
    
    def fit(self, X_train, y_train):
        self.arbol = construir_arbol(pd.DataFrame(X_train, columns=self.columnas), y_train)
        return self
    
    def predict(self, X_test):
        predictions = []
        for x_t in X_test:
            x_t_df = pd.DataFrame([x_t], columns=self.columnas).iloc[0]
            prediction = predecir(self.arbol, x_t_df) 
            print(x_t, "predicción ->", prediction)
            predictions.append(prediction)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        accuracy = sum(y_i == y_j for (y_i, y_j) in zip(y_pred, y_test)) / len(y_test)
        return accuracy
        

# Ejemplo de uso
clf = MiClasificadorArbol()

# Tomar en cuenta que sklearn espera numpy arrays:
clf.fit(np.array(X), y)
clf.score(np.array(X), y)


['Sol' 'Calor' 'Alta' 'Debil'] predicción -> 0.0
['Sol' 'Calor' 'Alta' 'Fuerte'] predicción -> 0.0
['Nublado' 'Calor' 'Alta' 'Debil'] predicción -> 0.0
['Lluvia' 'Templado' 'Alta' 'Debil'] predicción -> 0.0
['Lluvia' 'Frio' 'Normal' 'Debil'] predicción -> 0.0
['Lluvia' 'Frio' 'Normal' 'Fuerte'] predicción -> 0.0
['Nublado' 'Frio' 'Normal' 'Fuerte'] predicción -> 0.0
['Sol' 'Templado' 'Alta' 'Debil'] predicción -> 0.0
['Sol' 'Frio' 'Normal' 'Debil'] predicción -> 0.0
['Lluvia' 'Templado' 'Normal' 'Debil'] predicción -> 0.0
['Sol' 'Templado' 'Normal' 'Fuerte'] predicción -> 0.0
['Nublado' 'Templado' 'Alta' 'Fuerte'] predicción -> 0.0
['Nublado' 'Calor' 'Normal' 'Debil'] predicción -> 0.0
['Lluvia' 'Templado' 'Alta' 'Fuerte'] predicción -> 0.0


/usr/local/Cellar/jupyter/1.0.0_5/libexec/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


0.0